In [2]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn import decomposition
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from matplotlib import pyplot as plt
import matplotlib.style as style

import plotly
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
plotly.plotly.sign_in('spersad', 'oNkuP1yzbpN734Ag8M9P')
import plotly.graph_objs as go

from IPython.core.display import display, HTML


In [3]:

data = pd.read_csv('data_CNA.xls.txt',sep='\t')
labels= pd.read_csv('OLD_DATA/data_clinical_sample_clean.tsv',sep='\t')

data.set_index('Hugo_Symbol', inplace=True)
labels.set_index('SAMPLE_ID', inplace=True)

data.drop(['Entrez_Gene_Id'], axis=1,inplace=True)
labels.drop(['Unnamed: 0'], axis=1,inplace=True)
data = data.transpose() # Convert data to matrix, rows are tumour samples

labels = labels['ONCOTREE_CODE']
result = pd.concat([data, labels], axis=1)
result.head()

labels=result['ONCOTREE_CODE']

X = data.as_matrix() 
Y = labels

print(X.shape)

(103, 23109)


In [121]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import time

randomize = False
if randomize:
    random_indices = np.random.permutation(len(X))
    # Randomized data
    X_r = X[random_indices]
    Y_r = Y[random_indices]


def run_regression(X_r, Y_r, penalty = 'l2', per_train=0.6, per_val=0.1):
    
    # Select a training, test and validation set

    n_train = int(per_train*X.shape[0])
    n_val = int(per_val*X.shape[0])


    X_train, y_train = X_r[:n_train], Y_r[:n_train]
    X_val, y_val = X_r[n_train:n_train+n_val], Y_r[n_train:n_train+n_val]
    X_test, y_test = X_r[n_train+n_val:], Y_r[n_train+n_val:]

    # How many of each cancer type are in the train, test and validations set?
    tr_count = y_train.value_counts()/np.sum( y_train.value_counts())
    val_count = y_val.value_counts()/np.sum(y_val.value_counts())
    te_count = y_test.value_counts()/np.sum(y_test.value_counts())


    # Plot the distribution of test, train and val points. Will throw a KeyError if any
    try:
        trace1 = go.Bar(
            x=['BREAST','IDC', 'MDLC', 'ILC'],
            y=[tr_count[i] for i in range(4)],
            name='Training Set'
        )
        trace2 = go.Bar(
            x=['BREAST','IDC', 'MDLC', 'ILC'],
            y=[val_count[i] for i in range(4)],
            name='Validation Set'
        )

        trace3 = go.Bar(
            x=['BREAST','IDC', 'MDLC', 'ILC'],
            y=[te_count[i] for i in range(4)],
            name='Test Set'
        )

        data = [trace1, trace2, trace3]
        layout = go.Layout(
            barmode='group',
            title = 'Distribution of Categories over Data Sets',
            xaxis = dict(title = 'Categories'),
            yaxis = dict(title = 'Percentage of Data Set'),
        )

        fig = go.Figure(data=data, layout=layout)
        iplot(fig, filename='grouped-bar')

    except KeyError:
        print(tr_count)
        print(val_count)
        print(te_count)
        print('One of the data sets does not contain all the categories!')
        assert False


    t0 = time.clock()

    train_samples, n_features = X_train.shape
    n_classes = np.unique(Y).shape[0]

    print('Dataset 2CN, train_samples=%i, n_features=%i, n_classes=%i'
          % (train_samples, n_features, n_classes))

    models = {'ovr': {'name': 'One versus Rest', 'iters': [1, 3, 7, 11, 13, 15]},
              'multinomial': {'name': 'Multinomial', 'iters': [1, 3, 7, 11, 13, 15]}}

    # Solver which handles multiclass and l1 penalty
    solver = 'newton-cg'

    if penalty == 'l1':
        print('Using l1 regression')
        models = {'ovr': {'name': 'One versus Rest', 'iters': [1, 3, 7, 11, 13, 15]}}
        solver = 'liblinear'

    best_val_accuracy = 0
    best_model = None 
    
    for model in models:
        # Add initial chance-level values for plotting purpose
        test_accuracies = [1 / n_classes]
        tr_accuracies = [1/n_classes]
        val_accuracies = [1/n_classes]
        times = [0]
        densities = [1]

        model_params = models[model]

        # Small number of epochs for fast runtime
        for this_max_iter in model_params['iters']:
            print('[model=%s, solver=%s] Number of epochs: %s' %
                  (model_params['name'], solver, this_max_iter))
            lr = LogisticRegression(solver=solver,
                                    multi_class=model,
                                    C=1,
                                    penalty=penalty,
                                    fit_intercept=True,
                                    max_iter=this_max_iter,
                                    random_state=42,
                                    )
            t1 = time.clock()
            lr.fit(X_train, y_train)
            train_time = time.clock() - t1

            y_pred = lr.predict(X_test)
            accuracy = np.sum(y_pred == y_test) / y_test.shape[0]
            density = np.mean(lr.coef_ != 0, axis=1) * 100
            tr_accuracy = np.sum(y_train == lr.predict(X_train)) / y_train.shape[0]
            val_accuracy = np.sum(y_val == lr.predict(X_val)) / y_val.shape[0]

            test_accuracies.append(accuracy)
            densities.append(density)
            tr_accuracies.append(tr_accuracy)
            val_accuracies.append(val_accuracy)
            times.append(this_max_iter)
            
            if val_accuracy>best_val_accuracy:
                best_model = lr
                
        models[model]['times'] = times
        models[model]['densities'] = densities
        models[model]['test_accuracies'] = test_accuracies
        models[model]['tr_accuracies'] = tr_accuracies
        models[model]['val_accuracies'] = val_accuracies

        print('Test accuracy for model %s: %.4f' % (model, test_accuracies[-1]))
        print('Train accuracy for model %s: %.4f' % (model, tr_accuracies[-1]))
        print('Validation accuracy for model %s: %.4f' % (model, val_accuracies[-1]))
        print('%% non-zero coefficients for model %s, '
              'per class:\n %s' % (model, densities[-1]))
        print('Run time (%i epochs) for model %s:'
              '%.2f' % (model_params['iters'][-1], model, times[-1]))

    fig = plt.figure()
    ax = fig.add_subplot(111)

    for model in models:
        data = []
        for accuracy in ['tr_accuracies', 'val_accuracies','test_accuracies']:
            trace = go.Scatter(
                                x = models[model]['times'],
                                y = models[model][accuracy],
                                name = accuracy
                            )

            data.append(trace)
        layout = go.Layout(
            title = 'Accuracy for {0} model'.format(model),
            xaxis = dict(title = 'Iterations'),
            yaxis = dict(title = 'Accuracy'),
        )

        fig = go.Figure(data=data, layout=layout)
        iplot(fig)

    run_time = time.clock() - t0
    print('Example run in %.3f s' % run_time)

    from sklearn.metrics import confusion_matrix
    c = confusion_matrix(y_test, best_model.predict(X_test))

    trace = go.Heatmap(z=(c.T/np.sum(c,axis=1)).T,                   
                       x=['BREAST','IDC', 'MDLC', 'ILC'],
                       y=['BREAST','IDC', 'MDLC', 'ILC'])
    data=[trace]

    layout = go.Layout(
        title = 'Confusion Matrix for Test Predictions',
        xaxis = dict(title = 'Predicted'),
        yaxis = dict(title = 'Truth'),
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
    c = confusion_matrix(y_train, best_model.predict(X_train))
    trace = go.Heatmap(z=(c.T/np.sum(c,axis=1)).T,                   
                       x=['BREAST','IDC', 'MDLC', 'ILC'],
                       y=['BREAST','IDC', 'MDLC', 'ILC'])
    data=[trace]

    layout = go.Layout(
        title = 'Confusion Matrix for Training Predictions',
        xaxis = dict(title = 'Predicted'),
        yaxis = dict(title = 'Truth'),
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


    return best_model, lr


In [103]:
lr, last = run_regression(X_r, Y_r, penalty = 'l2')

Dataset 2CN, train_samples=51, n_features=23109, n_classes=4
[model=One versus Rest, solver=newton-cg] Number of epochs: 1
[model=One versus Rest, solver=newton-cg] Number of epochs: 3


C:\Users\Sitara\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning:

newton-cg failed to converge. Increase the number of iterations.



[model=One versus Rest, solver=newton-cg] Number of epochs: 7
[model=One versus Rest, solver=newton-cg] Number of epochs: 11
[model=One versus Rest, solver=newton-cg] Number of epochs: 13
[model=One versus Rest, solver=newton-cg] Number of epochs: 15
Test accuracy for model ovr: 0.5952
Train accuracy for model ovr: 1.0000
Validation accuracy for model ovr: 0.7000
% non-zero coefficients for model ovr, per class:
 [ 100.  100.  100.  100.]
Run time (15 epochs) for model ovr:15.00
[model=Multinomial, solver=newton-cg] Number of epochs: 1
[model=Multinomial, solver=newton-cg] Number of epochs: 3
[model=Multinomial, solver=newton-cg] Number of epochs: 7
[model=Multinomial, solver=newton-cg] Number of epochs: 11
[model=Multinomial, solver=newton-cg] Number of epochs: 13
[model=Multinomial, solver=newton-cg] Number of epochs: 15
Test accuracy for model multinomial: 0.5952
Train accuracy for model multinomial: 1.0000
Validation accuracy for model multinomial: 0.7000
% non-zero coefficients fo

C:\Users\Sitara\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Example run in 6.388 s


In [101]:
lr = run_regression(X_r, Y_r, penalty = 'l1')

Dataset 2CN, train_samples=51, n_features=23109, n_classes=4
Using l1 regression
[model=One versus Rest, solver=liblinear] Number of epochs: 1
[model=One versus Rest, solver=liblinear] Number of epochs: 3
[model=One versus Rest, solver=liblinear] Number of epochs: 7
[model=One versus Rest, solver=liblinear] Number of epochs: 11
[model=One versus Rest, solver=liblinear] Number of epochs: 13
[model=One versus Rest, solver=liblinear] Number of epochs: 15
Test accuracy for model ovr: 0.5000
Train accuracy for model ovr: 1.0000
Validation accuracy for model ovr: 0.7000
% non-zero coefficients for model ovr, per class:
 [ 0.30291229  0.2120386   0.18607469  0.22934787]
Run time (15 epochs) for model ovr:15.00


C:\Users\Sitara\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Example run in 1.775 s


In [106]:
imputed = 'PATIENT_DATA_ALL_2.csv'
non_imputed = 'filename_2.txt'

merged = pd.read_csv(imputed, sep='\t', )
labels = pd.read_csv('OLD_DATA/data_clinical_sample_clean.tsv', sep='\t')

labels.set_index('SAMPLE_ID', inplace=True)
labels.drop(['Unnamed: 0'], axis=1, inplace=True)

merged.drop('PATIENT_ID', axis=1,inplace=True)
merged.drop('SAMPLE_ID', axis=1,inplace=True)
merged.drop(['Unnamed: 0'], axis=1,inplace=True)

labels = labels['ONCOTREE_CODE']
result = pd.concat([merged, labels], axis=1)


C:\Users\Sitara\Anaconda3\lib\site-packages\pandas\indexes\range.py:432: RuntimeWarning:

unorderable types: int() < str(), sort order is undefined for incomparable objects



In [120]:

# random_indices = np.random.permutation(len(merged))

# X_randomized = merged.iloc[random_indices]
# Y_randomized = labels[:-1].iloc[random_indices]


sample_lr_l1 = run_regression(X_randomized, Y_randomized, penalty = 'l2')

Dataset 2CN, train_samples=61, n_features=106, n_classes=4
[model=One versus Rest, solver=newton-cg] Number of epochs: 1
[model=One versus Rest, solver=newton-cg] Number of epochs: 3
[model=One versus Rest, solver=newton-cg] Number of epochs: 7
[model=One versus Rest, solver=newton-cg] Number of epochs: 11


C:\Users\Sitara\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning:

newton-cg failed to converge. Increase the number of iterations.



[model=One versus Rest, solver=newton-cg] Number of epochs: 13
[model=One versus Rest, solver=newton-cg] Number of epochs: 15
Test accuracy for model ovr: 0.6129
Train accuracy for model ovr: 0.7213
Validation accuracy for model ovr: 0.5000
% non-zero coefficients for model ovr, per class:
 [ 79.24528302  79.24528302  79.24528302  79.24528302]
Run time (15 epochs) for model ovr:15.00
[model=Multinomial, solver=newton-cg] Number of epochs: 1
[model=Multinomial, solver=newton-cg] Number of epochs: 3
[model=Multinomial, solver=newton-cg] Number of epochs: 7
[model=Multinomial, solver=newton-cg] Number of epochs: 11
[model=Multinomial, solver=newton-cg] Number of epochs: 13
[model=Multinomial, solver=newton-cg] Number of epochs: 15
Test accuracy for model multinomial: 0.6129
Train accuracy for model multinomial: 0.7049
Validation accuracy for model multinomial: 0.5000
% non-zero coefficients for model multinomial, per class:
 [ 79.24528302  79.24528302  79.24528302  79.24528302]
Run time (

C:\Users\Sitara\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Example run in 1.083 s
